# Task 1: XOR

In [14]:
# Import modules
from __future__ import print_function
import tensorflow as tf
import numpy as np
from numpy.random import shuffle
import time
import matplotlib.pyplot as plt

# Plot configurations
% matplotlib inline

# Notebook auto reloads code. (Ref: http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython)
% load_ext autoreload
% autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Task 1, Part 1: Backpropagation through time (BPTT)

**Question:** Consider a simple RNN network shown in the following figure, where __ _wi, wh, b, a, c_ __ are the scalar parameters of the network. The loss function is the **mean squared error (MSE)**. Given input (x0, x1) = (1, 0), ground truth (g1, g2) = (1, 1), h0 = 0, (wi, wh, b, a, c) = (1, 1, 1, 1, 1), compute __ _(dwi, dwh, db, da, dc)_ __, which are the gradients of loss with repect to 5 parameters __ _(wi, wh, b, a, c)_ __.

![bptt](./img/bptt.png)

<span style="color:red">TODO:</span>

Answer the above question. 

* **[fill in here: Enter your derivations and the computational process]**
* You can use LATEX to edit the equations, and Jupyter notebook can recognize basic LATEX syntax. Alternatively, you can edit equations in some other environment and then paste the screenshot of the equations here.

## Task 1, Part 2: Use tensorflow modules to create XOR network

In this part, you need to build and train an XOR network that can learn the XOR function. It is a very simple implementation of RNN and will give you an idea how RNN is built and how to train it.

### XOR network

XOR network can learn the XOR $\oplus$ function

As shown in the figure below, and for instance, if input $(x0, x1, x2)$=(1,0,0), then output $(y1, y2, y3)$=(1,1,1). That is, $y_n = x_0\oplus x_1 \oplus ... \oplus x_{n-1}$

![xor_net](./img/xor.png)

### Create data set
This function provides you the way to generate the data which is required for the training process. You should utilize it when building your training function for the LSTM. Please read the source code for more information.

In [15]:
from ecbm4040.xor.utils import create_dataset

### Build a network using a Tensorlow LSTMCell
This section shows an example how to build a RNN network using an LSTM cell. LSTM cell is an inbuilt class in tensorflow which implements the real behavior of the LSTM neuron. 

Reference: [TensorFlow LSTM cell](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/LSTMCell)

In [16]:
from tensorflow.contrib.rnn import LSTMCell

tf.reset_default_graph()

# Input shape: (num_samples, seq_length, input_dimension)
# Output shape: (num_samples, output_ground_truth), and output_ground_truth is 0/1. 
input_data = tf.placeholder(tf.float32,shape=[None,None,1])
output_data = tf.placeholder(tf.int64,shape=[None,None])

# define LSTM cell
lstm_units = 64
cell = LSTMCell(lstm_units,num_proj=2,state_is_tuple=True)

# create LSTM network: you can also choose other modules provided by tensorflow, like static_rnn etc.
out,_ = tf.nn.dynamic_rnn(cell,input_data,dtype=tf.float32)
pred = tf.argmax(out,axis=2)

# loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=output_data,logits=out))
#init = tf.global_variables_initializer()

# optimization
optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

# accuracy
correct_num = tf.equal(output_data,pred)
accuracy = tf.reduce_mean(tf.cast(correct_num,tf.float32))

### Training 

<span style='color:red'>TODO:</span> 
1. Build your training funciton for RNN; 
2. Plot the cost during the traning

In [17]:
# YOUR TRAINING AND PLOTTING CODE HERE
seq = 8
num = 80000
x_train, y_train = create_dataset(num, seq)
#x_train, y_train = create_dataset(num, seq)
loss_hist = []
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    #sess.run(init)
    for e in range(700):
        sess.run(optimizer, feed_dict={input_data: x_train, output_data: y_train})
        #x_train, y_train = create_dataset(num, seq)
        cost = sess.run(loss, feed_dict={input_data: x_train, output_data: y_train})
        loss_hist.append(cost)
        print('epoch {}: cost = {}'.format(e+1, cost))
        
    print('Final Loss = {}'.format(cost))
        


epoch 1: cost = 0.6962832808494568
epoch 2: cost = 0.6957697868347168
epoch 3: cost = 0.6953020095825195
epoch 4: cost = 0.6951442360877991
epoch 5: cost = 0.6948001384735107
epoch 6: cost = 0.6944846510887146
epoch 7: cost = 0.6944009065628052
epoch 8: cost = 0.6941502094268799
epoch 9: cost = 0.6941158771514893
epoch 10: cost = 0.6939509510993958
epoch 11: cost = 0.6938799023628235
epoch 12: cost = 0.693794846534729
epoch 13: cost = 0.6938340663909912
epoch 14: cost = 0.6937688589096069
epoch 15: cost = 0.6935593485832214
epoch 16: cost = 0.6934341788291931
epoch 17: cost = 0.6931628584861755
epoch 18: cost = 0.693203866481781
epoch 19: cost = 0.6930791735649109
epoch 20: cost = 0.693057119846344
epoch 21: cost = 0.692785382270813
epoch 22: cost = 0.6926944851875305
epoch 23: cost = 0.6926962733268738
epoch 24: cost = 0.6927034258842468
epoch 25: cost = 0.6922024488449097
epoch 26: cost = 0.6920996904373169
epoch 27: cost = 0.6919618844985962
epoch 28: cost = 0.6918278336524963
epoch

KeyboardInterrupt: 

## Task 1, Part 3 :  Build your own LSTMCell
In this part, you need to build your own LSTM cell to achieve the LSTM functionality. 

<span style="color:red">TODO:</span> 
1. Finish class **MyLSTMCell** in ecbm4040/xor/rnn.py;
2. Write the training function for your RNN;
3. Plot the cost during training.

In [7]:
from ecbm4040.xor.rnn import MyLSTMCell

# recreate xor netowrk with your own LSTM cell
tf.reset_default_graph()

#Input shape: (num_samples,seq_length,input_dimension)
#Output shape: (num_samples, output_ground_truth), and output_ground_truth is 0/1. 
input_data = tf.placeholder(tf.float32,shape=[None,None,1])
output_data = tf.placeholder(tf.int64,shape=[None,None])

# recreate xor netowrk with your own LSTM cell
lstm_units = 64
cell = MyLSTMCell(lstm_units,num_proj=2)

# create LSTM network: you can also choose other modules provided by tensorflow, like static_rnn etc.
out,_ = tf.nn.dynamic_rnn(cell,input_data,dtype=tf.float32)
pred = tf.argmax(out,axis=2)

# loss function
loss = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(labels=output_data,logits=out))
# optimization
optimizer = tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)
# accuracy
correct = tf.equal(output_data,pred)
accuracy = tf.reduce_mean(tf.cast(correct,tf.float32))

TypeError: cell must be an instance of RNNCell

### Training

In [ ]:
# YOUR TRAINING AND PLOTTING CODE HERE

In [2]:
import tensorflow as tf
print (tf.__file__)

C:\Users\zoran\Anaconda3\envs\dlWorksA3\lib\site-packages\tensorflow\__init__.py
